# 125个经典技术指标择时分析
## 广发证券 2020年1月3日

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time
import os

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed
import indicatorJ

# 基础函数
import utilsJ

## 策略

### 策略主体

In [30]:
class gf125(bt.Strategy):
    
    params = (
        ('printlog', False),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to buyprice and sellprice in the data[0] dataseries
        self.buyprice = None
        self.sellprice = None
        
        # Add indicators
        self.ER = indicatorJ.ER(self.data)
        self.DPO = indicatorJ.DPO(self.data)
        self.POS = indicatorJ.POS(self.data)
        self.TII = indicatorJ.TII(self.data)
        self.ADTM = indicatorJ.ADTM(self.data)
        self.MADisplaced = indicatorJ.MADisplaced(self.data)
        self.T3 = indicatorJ.T3(self.data)
        self.VMA = indicatorJ.VMA(self.data)

        #self.KDJ = indicatorJ.KDJ(self.data)
        self.MAAMT = indicatorJ.MAAMT(self.data)
        self.SROCVOL = indicatorJ.SROCVOL(self.data)

        self.ind_num = 10
        self.buy_votes = self.ER.lines.Buy + self.DPO.lines.Buy + self.POS.lines.Buy + self.TII.lines.Buy + \
                         self.ADTM.lines.Buy + self.MADisplaced.lines.Buy + self.T3.lines.Buy + self.VMA.lines.Buy + \
                         self.MAAMT.lines.Buy + self.SROCVOL.lines.Buy
        self.sell_votes = self.ER.lines.Sell + self.DPO.lines.Sell + self.POS.lines.Sell + self.TII.lines.Sell + \
                          self.ADTM.lines.Sell + self.MADisplaced.lines.Sell + self.T3.lines.Sell + self.VMA.lines.Sell + \
                          self.MAAMT.lines.Sell + self.SROCVOL.lines.Sell
        


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
#        if self.order:
#            return
        if self.buy_votes[0] >= self.ind_num // 2 and self.getposition(self.data).size == 0:
            lots = np.floor(self.broker.getcash() / (100*self.data.close[0]))*100
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.data.close[0], lots, self.getposition(self.data).size))
            self.order = self.buy(size=lots)
        elif self.sell_votes[0] >= self.ind_num // 2 and self.getposition(self.data).size != 0:
            self.log('Sell CREATE (Close), Price: %.2f, Current Position: %i' % (self.data.close[0], self.getposition(self.data).size))
            self.order = self.close()

## 单股回测/测试

In [53]:
s_date = datetime.date(2018,12,31)
e_date = datetime.date(2022,9,30)
stock_code = '300015.SZ'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(gf125, printlog=True)


    # Create stock Data Feed
    if os.path.exists('.\\Data\\'+stock_code+'.csv'):
        df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
                    converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    else:
        df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

    data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(200000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 200000.00
2020-01-09: BUY CREATE, Price: 18.78, Lots: 10600, Current Position: 0
2020-01-10: BUY EXECUTED, Price: 18.78, Lot:10600, Cash: 927, Value: 201538
2022-03-28: Sell CREATE (Close), Price: 22.33, Current Position: 10600
2022-03-29: SELL EXECUTED, Price: 22.33, Lot:10600, Cash: 237613, Value: 237613
2022-03-29: OPERATION PROFIT, GROSS 37613.04, NET 37613.04
2022-04-12: BUY CREATE, Price: 25.27, Lots: 9400, Current Position: 0
2022-04-13: BUY EXECUTED, Price: 25.27, Lot:9400, Cash: 88, Value: 241797
Final Portfolio Value: 269586.20
Net Profit: 34.79%


# 沪深300回测/测试

In [35]:
s_date = datetime.date(2019,12,31)
e_date = datetime.date(2021,12,31)
stock_index = '000300.SH'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'

if __name__ ==  '__main__':

    pro = ts.pro_api(token)
    index_list = np.unique(pro.index_weight(index_code=stock_index, 
                              start_date=s_date.strftime('%Y%m%d'), 
                              end_date=e_date.strftime('%Y%m%d')).con_code).tolist()
    
    profit_dict = {}

    for stock_code in index_list:

        # Create a cerebro entity
        cerebro = bt.Cerebro()

        # Add a strategy
        strats = cerebro.addstrategy(gf125, printlog=False) 


        # Create stock Data Feed
        if os.path.exists('.\\Data\\'+stock_code+'.csv'):
            df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
                        converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
        else:
            df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

        data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

        # Add the index Data Feed to Cerebo
        cerebro.adddata(data)

        # Set cash inside the strategy
        cerebro.broker = bt.brokers.BackBroker(coc=True)   
        cerebro.broker.setcash(20000)

        # Set commission
        #cerebro.broker.setcommission()

        # Print out the starting conditions
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Run over everything
        cerebro.run()

        # Print out the final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
        profit_dict[stock_code] = (final_value - start_value) / start_value * 100

IndexError: array assignment index out of range

In [44]:
list(profit_dict.values())

[-12.7025,
 -27.656299999999973,
 13.74120000000001,
 -14.448000000000011,
 5.970000000000001,
 -4.65,
 -7.2375,
 10.229700000000012,
 140.25580000000002,
 -7.978899999999994,
 1.9150500000000101,
 21.545099999999984,
 26.168200000000013,
 33.735,
 0.0,
 34.2618,
 -16.7425,
 -15.285999999999985,
 -34.856549999999984,
 0.0,
 -26.256450000000005,
 -12.4,
 19.634449999999998,
 112.32,
 -5.065300000000007,
 -7.544500000000006,
 6.251499999999996,
 41.00195,
 0.13719999999999344,
 0.0,
 -19.238250000000008,
 0.0,
 4.925,
 26.08,
 -34.9304,
 3.06,
 -50.14344999999999,
 17.0,
 13.60379999999999,
 -10.508250000000007,
 25.595599999999997,
 -22.15,
 23.16939999999999,
 -25.55,
 5.175,
 0.0,
 0.0,
 0.0,
 3.16,
 0.0,
 39.37700000000001,
 35.3814,
 0.0,
 -28.363200000000006,
 4.380750000000008,
 -40.85894999999998,
 21.44699999999999,
 12.836849999999995,
 5.273799999999992,
 -7.38255000000001,
 -1.78,
 -11.174799999999996,
 -6.345000000000001,
 0.0,
 5.2,
 4.77,
 0.0,
 22.80835000000001,
 -9.6100